In [1]:
import gym

In [2]:
from gym.envs.box2d import CarRacing
from stable_baselines.common.vec_env import DummyVecEnv

/home/pablo/anaconda3/envs/ppo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pablo/anaconda3/envs/ppo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pablo/anaconda3/envs/ppo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pablo/anaconda3/envs/ppo/lib/python3.7/site-packages

In [3]:
#std env from Mike
if __name__=='__main__':
    env0 = lambda :  CarRacing(
        grayscale=1,
        show_info_panel=1,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_tracks=1,
        )
    
    #env = getattr(environments, env)
    env0 = DummyVecEnv([env0])

    obs = env0.reset()


Track generation: 1102..1383 -> 281-tiles track
retry to generate track (normal if there are not many of this messages)
Track generation: 1200..1504 -> 304-tiles track


In [4]:
print(env0.action_space)
print(env0.observation_space)

Discrete(5)
Box(96, 96, 4)


In [5]:
try:
    print(env0.action_space.high)
    print(env0.action_space.low)
except:
    print('discrete')
    print(env0.action_space.sample())
    

discrete
4


print(env0.observation_space.high)
print(env0.observation_space.low)

In [6]:
env0.close()

In [7]:
from stable_baselines.common.policies import CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2 #, PPO1


In [8]:
#training env
if __name__=='__main__':
    env1 = lambda :  CarRacing(
        grayscale=1,
        show_info_panel=0,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_tracks=1,
        max_time_out=2.0,
        max_step_reward=1,
        allow_outside=True,
        )


In [9]:
# multiprocess environment
paralelo=1

if paralelo==1:
    env1 = DummyVecEnv([env1])
    #env1 = gym.make(env1)
else:
    env1 = make_vec_env(env1, n_envs=paralelo)

obs = env1.reset()
print(obs.shape)

Track generation: 1031..1293 -> 262-tiles track
(1, 96, 96, 4)


In [ ]:
batch=1000
updates=250
cont=False

if not cont:
    model = PPO2(CnnLnLstmPolicy, env1, verbose=1, n_steps=batch, n_cpu_tf_sess=2, nminibatches=paralelo) 
    #, max_grad_norm=1)
    #model = PPO2(CnnLstmPolicy, env1, verbose=1, n_steps=batch, n_cpu_tf_sess=2, nminibatches=paralelo)

model.learn(total_timesteps=updates*batch*paralelo)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
import pickle
file='ppo2_v1_mike_lnlstm_4f_bs1000'

model.save(file, cloudpickle=True)
param_list=model.get_parameter_list()
param=model.get_parameters()

f = open(file+'_param.pkl',"wb")
#pickle.dump(param_list,f)
pickle.dump(param,f)
f.close()

In [ ]:
#batch=512
updates=250
cont=True

if not cont:
    model = PPO2(CnnLnLstmPolicy, env1, verbose=1, n_steps=batch, n_cpu_tf_sess=2, nminibatches=paralelo) 
    #, max_grad_norm=1)
    #model = PPO2(CnnLstmPolicy, env1, verbose=1, n_steps=batch, n_cpu_tf_sess=2, nminibatches=paralelo)

model.learn(total_timesteps=updates*batch*paralelo)


In [ ]:
#import pickle

model.save(file+'II', cloudpickle=True)
param_list=model.get_parameter_list()
param=model.get_parameters()

f = open(file+'II_param.pkl',"wb")
#pickle.dump(param_list,f)
pickle.dump(param,f)
f.close()

In [ ]:
env1.close()

model2 = PPO2.load('ppo2_v1_mike_lstm_para4.pkl')
#model.set_env(env)


In [ ]:
#play env
if __name__=='__main__':
    env2 = lambda :  CarRacing(
        grayscale=1,
        show_info_panel=1,
        discretize_actions="hard",
        frames_per_state=4,
        num_lanes=1,
        num_tracks=1,
        )

    #env = getattr(environments, env)
    env2 = DummyVecEnv([env2])

    obs = env2.reset()
    
    done = False
    pasos = 0
    _states=None
    while not done:
        action, _states = model.predict(obs, _states)
        obs, reward, done, info = env2.step(action)
        env2.render()
        pasos+=1


In [ ]:
env2.close()

In [ ]:
obs.shape

In [ ]:
print(reward, done, pasos)
print(info)

In [ ]:
print(action, _states)